# EM Algorithm - Univariate Gaussian Mixture Model 

In [19]:
import numpy as np
from tqdm import tqdm 
import matplotlib.pyplot as plt

## Estimating 3 Component GMM

In [20]:
N = 1500
D = 2
K = 3
mu_1, sigma_1 = ( np.array([-4, -5]), np.array([[1, 0], [0, 1]]) )
mu_2, sigma_2 = ( np.array([0, 0]), np.array([[1, 0], [0, 1]]) )
mu_3, sigma_3 = ( np.array([4, 5]), np.array([[1, 0], [0, 1]]) )

priors = [0.2, 0.5, 0.3]

X1 = np.random.multivariate_normal(mean=mu_1, cov=sigma_1, size=int(priors[0]*N))
X2 = np.random.multivariate_normal(mean=mu_2, cov=sigma_2, size=int(priors[1]*N))
X3 = np.random.multivariate_normal(mean=mu_3, cov=sigma_3, size=int(priors[2]*N))
X = np.concatenate((X1, X2, X3))
X1.shape, X2.shape, X3.shape, X.shape

((300, 2), (750, 2), (450, 2), (1500, 2))

Probability Density Function of Multivariate Gaussian

In [21]:
def pdf(x, mu, covariance):
    """pdf of the multivariate normal distribution."""
    x_m = x - mu
    d = x.shape[-1]
    return (1. / (np.sqrt((2 * np.pi)**d * np.linalg.det(covariance))) * 
            np.exp(-(np.linalg.solve(covariance, x_m).T.dot(x_m)) / 2))

Random Initialization

In [25]:
lambdas = np.ones((K))/K
means = np.array([np.random.choice(X[:, i], K) for i in range(D)]).transpose(-1, 0) #(K, D)
covariances = np.array([np.identity(D)/K for _ in range(K)])
print('Means: ', means, '\n\n', 'Cov: ', covariances, '\n\n', 'Lambdas: ', lambdas)

Means:  [[-0.04859585 -0.40988337]
 [-5.59151489  0.49644395]
 [-0.90279152  4.84687693]] 

 Cov:  [[[0.33333333 0.        ]
  [0.         0.33333333]]

 [[0.33333333 0.        ]
  [0.         0.33333333]]

 [[0.33333333 0.        ]
  [0.         0.33333333]]] 

 Lambdas:  [0.33333333 0.33333333 0.33333333]


Iterative EM algorithm (till convergence)

In [26]:
ll_hist = []
for iteration in range(50):
    G = np.zeros((K, N)) #Gamma

    #E-Step
    for k in range(K):
        G[k, :] = np.array([pdf(X_i, means[k], covariances[k]) * lambdas[k] for X_i in X])
    ll_hist.append(G.sum())
    G = G / np.sum(G, axis=0)
  
    #M-Step
    means = np.matmul(G, X)
    for k in range(means.shape[0]):
        means[k] /= np.sum(G, axis=1)[k]
    means += 1e-18
    print(means.shape)

    for k in range(K):
        numerator = 0
        for i in range(N):
            Y = (X[i] - means[k]).reshape(D, 1)
            numerator += G[k, i] * np.matmul(Y, Y.T)
        covariances[k] = numerator / np.sum(G[k]) 
        lambdas[k] = np.mean(G[k])
    
    print(f'Iteration {iteration}: \nmeans - {means}, \nvariances - {covariances}, \nlambdas = {lambdas}\n')

print(f'Results\n: means - {means}, \nvariances - {covariances}, \nlambdas = {lambdas}\n')

(3, 2)
Iteration 0: 
means - [[-0.25451317 -0.60342661]
 [-4.62508074 -4.8776498 ]
 [ 3.83796422  4.91038822]], 
variances - [[[2.10399254 1.91938299]
  [1.91938299 3.87044875]]

 [[0.54868082 0.16816517]
  [0.16816517 1.08623486]]

 [[1.41962449 0.32157223]
  [0.32157223 0.94734746]]], 
lambdas = [0.56624651 0.13166381 0.30208968]

(3, 2)
Iteration 1: 
means - [[-0.14526279 -0.32953484]
 [-4.42182435 -5.00323779]
 [ 3.98606318  4.9622608 ]], 
variances - [[[1.80390939 1.42559109]
  [1.42559109 3.22242546]]

 [[0.71978568 0.03685598]
  [0.03685598 1.02220872]]

 [[0.91254118 0.00652483]
  [0.00652483 0.86352202]]], 
lambdas = [0.55651615 0.15230664 0.29117721]

(3, 2)
Iteration 2: 
means - [[-0.0080326  -0.0832386 ]
 [-4.2326862  -5.06393358]
 [ 4.00284802  4.96984847]], 
variances - [[[ 1.41229331  0.72983576]
  [ 0.72983576  2.02633139]]

 [[ 0.88563289 -0.0534449 ]
  [-0.0534449   1.06111618]]

 [[ 0.87232253 -0.01603521]
  [-0.01603521  0.85549161]]], 
lambdas = [0.53208818 0.17756

In [27]:
import plotly.express as px 
fig = px.line(x=list(range(len(ll_hist))), y=ll_hist)
fig.update_layout(
    title='Likelihood vs time',
    xaxis_title="iteration",
    yaxis_title="likelihood",
)
fig.show()